# Table of contents

1. [Introduction](#introduction)
2. [Card](#card)
    - [Attributes](#attributes)
    - [Card Class](#card-class)
3. [Deck](#deck)
    - [Important Parts of the Deck Class](#important-parts-of-the-deck-class)
    - [Card Colors](#card-colors)
4. [Core game Mechanics](#core-game-mechanics)
    - [Player Class](#player-class)
    - [Methods of the Player Class](#methods-of-the-player-class)
5. [Game logic](#game-logic)
    - [The game class](#the-game-class)
    - [Methods](#methods)
```

## 🃏 Card
- **Attributes**:
  - `rank`: The rank of the card (e.g., Ace, 2, 3, 4, ..., Queen, King)
  - `suit`: The suit of the card (e.g., Hearts, Diamonds, Clubs, Spades)

Using both the `__repr__` and `__str__` methods will give us the best of both worlds: a clear, user-friendly string for players to read, and a more detailed string for developers (me xD) to debug.

### 🃏  Card Class

In this section, we will dive into the creation of the `Card` class, which represents a single playing card. Here's what we'll cover:

- 🛠️ **Constructor (`__init__` method)**: This special method is called automatically when a new instance of the class is created. It initializes the card with a `rank` and a `suit`.

- 📦 **Self Parameter**: A reference to the current instance of the class, used to access variables that belong to the class.

- 🃏 **Attributes**: The `rank` and `suit` parameters are used to initialize the `Card` object.

- 🖨️ **String Representation (`__str__` method)**: Another special method that is called when the `print()` function is used on a `Card` object. This method returns a string representation of the `Card` object.

- 🔍 **Developer Representation (`__repr__` method)**: This method is called when the `repr()` function is used on a `Card` object. It returns a detailed string representation of the `Card` object, useful for debugging.

Let's get started! 🚀

In [12]:
# Step 1: Define the Card class
class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
    
    def __repr__(self):
        return f"Card(value='{self.value}', suit='{self.suit}')"
    
    def __str__(self):
        return f"Card: {self.value} of {self.suit}"

## 🃏 Deck
The `Deck` class represents a deck of playing cards. 🃏

### Important Parts of the `Deck` Class

1. **Attributes**:
   - `cards`: A list that holds all the card objects in the deck. 🗂️
   - `discard_pile`: A list that holds the cards that have been discarded. 🗑️

2. **Methods**:
   - `__init__()`: Initializes the deck by creating all 52 cards (values 1-10, J, Q, K, A) and storing them in the `cards` list. The deck is then shuffled. 🛠️
   - `draw_card()`: Draws a card from the top of the deck. If the deck is empty, it reshuffles the discard pile back into the deck. 🎴
   - `reshuffle()`: Reshuffles the discard pile back into the deck. 🔄
   - `add_to_discard(card)`: Adds a card to the discard pile. ➕🗑️
   - `top_discard()`: Returns the top card of the discard pile without removing it. 🔝🗑️

### Card Colors
- Each card has a `color` attribute that is determined by its suit:
  - Red for Hearts (♥) and Diamonds (♦)
  - Black for Spades (♠) and Clubs (♣)



In [13]:
import random

class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
        self.color = 'red' if suit in ['♥', '♦'] else 'black'
    
    def __repr__(self):
        return f"Card(value='{self.value}', suit='{self.suit}', color='{self.color}')"
    
    def __str__(self):
        return f"Card: {self.value} of {self.suit} ({self.color})"

class Deck:
    def __init__(self):
        # Create a standard deck of cards (values 1-10, J, Q, K, A for simplicity)
        self.cards = [Card(value, suit) for value in list(range(1, 11)) + ['J', 'Q', 'K', 'A'] 
                      for suit in ['♠', '♥', '♦', '♣']]
        random.shuffle(self.cards)
        self.discard_pile = []
        
    def draw_card(self):
        """Draws a card from the top of the deck."""
        if not self.cards:
            self.reshuffle()
        return self.cards.pop()
    
    def reshuffle(self):
        """Reshuffles the discard pile back into the deck."""
        self.cards = self.discard_pile
        self.discard_pile = []
        random.shuffle(self.cards)
    
    def add_to_discard(self, card):
        """Adds a card to the discard pile."""
        self.discard_pile.append(card)
    
    def top_discard(self):
        """Returns the top card of the discard pile without removing it."""
        return self.discard_pile[-1] if self.discard_pile else None

## Core game Mechanics

Here we will describe the player interactions and the flow of the game, and then, include the special rules and player actions:

### Player Class 🧑
- **Attributes**:
  - `name`: The name of the player.
  - `hand`: The list of cards in the player's hand.
  - `score`: The player's score.
  - `is_active`: A boolean value indicating whether the player is active or not.
  - `is_winner`: A boolean value indicating whether the player is the winner or not.




In [14]:
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = []  # Contains four cards: two face-up and two face-down
        self.score = 0
        self.is_active = True  # Indicates whether the player is active
        self.is_winner = False  # Indicates whether the player is the winner
        
    def draw_from_deck(self, deck):
        """Draws a card from the deck that is different from the cards in the player's hand."""
        while True:
            drawn_card = deck.draw_card()
            if all(drawn_card.value != card.value or drawn_card.suit != card.suit for card in self.hand):
                return drawn_card
    
    def draw_from_discard(self, deck):
        """Draws the top card from the discard pile."""
        return deck.draw_card()  # Fix: Remove the card from the discard pile
    
    def exchange_card(self, card_index, new_card):
        """Exchanges a card in the player's hand with a new card if it lowers the score."""
        current_card = self.hand[card_index]
        if self.card_value(new_card) < self.card_value(current_card):
            self.hand[card_index] = new_card
            return current_card
        else:
            return new_card

    def card_value(self, card):
        """Returns the value of a card for scoring purposes."""
        if card.value in ['J', 'Q']:
            return 10
        elif card.value == 'K':
            return 0 if card.color == 'red' else 20
        elif card.value == 'A':
            return 1  # Assign a value for Ace
        else:
            return int(card.value)  # Ensure integer comparison

    def calculate_score(self):
        """Calculates the score based on the card values and colors."""
        score = 0
        for card in self.hand:
            score += self.card_value(card)
        return score

### Methods of the Player Class:
- 🛠️ `__init__(name)`: Initializes the player with a name, an empty hand, a score of 0, and sets the player as active and not a winner.

- 🎴 `draw_from_deck(deck)`: Draws a card from the deck and returns it. This method interacts with the `Deck` class to get a card from the top of the deck. The player then decides whether to discard it or exchange it with a known card from their hand if the known card has a greater score.

- 🗑️ `draw_from_discard(deck)`: Draws the top card from the discard pile and returns it. This method interacts with the `Deck` class to get the top card from the discard pile.

- 🔄 `exchange_card(card_index, new_card)`: Exchanges a card in the player's hand with a new card. The method takes the index of the card to be exchanged and the new card to replace it. It returns the discarded card.

- 📊 `calculate_score()`: Calculates the player's score based on the values and colors of the cards in their hand. J and Q have a fixed score of 10. Red K has a score of 0, while Black K has a score of 20. Other cards contribute their face value to the score. The method iterates through each card in the player's hand to compute the total score.



## Game logic

## The `game` class:
### Attributes:
- `players`: A list of `Player` objects participating in the game.
- `deck`: An instance of the `Deck` class representing the deck of cards used in the game.
- `current_turn`: An integer representing the index of the player whose turn it is.
- `game_ended`: A boolean value indicating whether the game has ended.

In [15]:
class Game:
    def __init__(self, players):
        self.players = players
        self.deck = Deck()
        self.current_turn = 0
        self.game_ended = False
    
    def deal_cards(self):
        """Deals initial four cards to each player: 2 face-up and 2 face-down."""
        for player in self.players:
            # Each player gets two known and two unknown cards
            player.hand = [self.deck.draw_card() for _ in range(4)]
    
    def take_turn(self, player):
        """Handles a player's turn: draw, exchange, and manage special rules."""
        # Example turn logic: choose a card from either the deck or discard pile
        print(f"{player.name}'s turn.")
        # Let's assume player always draws from the deck for now
        drawn_card = player.draw_from_deck(self.deck)
        print(f"{player.name} drew a {drawn_card}")
        
        # Exchange the drawn card with one from the hand (simplified logic)
        discarded = player.exchange_card(0, drawn_card)  # Replace card at index 0 for example
        self.deck.add_to_discard(discarded)
        print(f"{player.name} discarded {discarded}")
    
    def play_round(self):
        """Plays one round of the game, where each player gets one turn."""
        for player in self.players:
            self.take_turn(player)
            if self.check_end_condition(player):
                break
    
    def check_end_condition(self, player):
        """Check if a player calls for the game end."""
        # Simplified end condition for now (can be expanded later)
        return False

### Methods:
- 🛠️ `__init__(players)`: Initializes the game with a list of players, a new deck, sets the current turn to 0, and marks the game as not ended.
- 🃏 `deal_cards()`: Deals initial four cards to each player: 2 face-up and 2 face-down.
- 🔄 `take_turn(player)`: Handles a player's turn, including drawing a card, exchanging it, and managing special rules.
- 🔁 `play_round()`: Plays one round of the game, where each player gets one turn.
- ❓ `check_end_condition(player)`: Checks if a player calls for the game end. This is a simplified end condition for now.

## The Setup
- We have two players: Alice and Bob.
- The game starts with both players receiving 4 cards each (2 face-up and 2 face-down).
- The goal is to simulate one round, where each player gets a turn to draw a card, exchange it, and the round ends.
### Step-by-Step Walkthrough of the Game Round

1. **Create the `Deck` and `Players`:**

    - A new `Deck` is created and shuffled.
    - Alice and Bob are instantiated as `Player` objects.
    - A `Game` object is created with Alice and Bob as participants.


In [16]:
deck = Deck()
alice = Player("Alice")
bob = Player("Bob")
players = [alice, bob]
game = Game(players)



2. **Deal Initial Cards:**

    - Each player is dealt 4 cards: 2 known (face-up) and 2 unknown (face-down). The known cards are visible only to the player holding them, while the unknown cards are hidden from everyone.



In [17]:
# Step 2: Deal initial cards
game.deal_cards()
print(f"Initial hands:")
print(f"{alice.name}'s hand: {alice.hand}")
print(f"{bob.name}'s hand: {bob.hand}")


Initial hands:
Alice's hand: [Card(value='6', suit='♠', color='black'), Card(value='K', suit='♠', color='black'), Card(value='4', suit='♠', color='black'), Card(value='6', suit='♣', color='black')]
Bob's hand: [Card(value='5', suit='♥', color='red'), Card(value='Q', suit='♠', color='black'), Card(value='7', suit='♥', color='red'), Card(value='9', suit='♣', color='black')]


3. **Alice's Turn:**

    - Alice decides to draw a card from the deck.
    - She draws a 5♠.
    - She decides to exchange this drawn card with one of her face-down cards (e.g., she replaces her second face-down card).
    - The card she exchanges is added to the discard pile.

In [18]:


print(f"{alice.name} draws a {drawn_card} from the deck.")

# Alice decides to replace a card in her hand with the drawn card based on the game logic
# For simplicity, let's replace the card with the highest value
def card_value_for_comparison(card):
    if isinstance(card.value, int):
        return card.value
    elif card.value in ['J', 'Q', 'K', 'A']:
        return {'J': 11, 'Q': 12, 'K': 13, 'A': 14}[card.value]
    else:
        return int(card.value)

highest_value_index = max(range(len(alice.hand)), key=lambda i: card_value_for_comparison(alice.hand[i]))
discarded_card = alice.exchange_card(highest_value_index, drawn_card)
deck.add_to_discard(discarded_card)

print(f"{alice.name} exchanges {discarded_card} for {drawn_card}.")
print(f"{alice.name}'s new hand: {alice.hand}")
print(f"Top of the discard pile: {deck.top_discard()}")

Alice draws a Card: 3 of ♥ (red) from the deck.
Alice exchanges Card: K of ♠ (black) for Card: 3 of ♥ (red).
Alice's new hand: [Card(value='6', suit='♠', color='black'), Card(value='3', suit='♥', color='red'), Card(value='4', suit='♠', color='black'), Card(value='6', suit='♣', color='black')]
Top of the discard pile: Card: K of ♠ (black)




4. **Bob's Turn:**

    - Bob decides to draw a card from the discard pile.
    - He draws Alice's discarded card (e.g., 5♠).
    - He decides to exchange this card with one of his face-down cards (e.g., he replaces his third face-down card).
    - His exchanged card is added to the discard pile.



In [19]:
print("\n-- Bob's Turn --")

# Use the already dealt hand for Bob
print(f"{bob.name}'s initial hand: {bob.hand}")

# Bob decides to draw a card from the discard pile
drawn_card = bob.draw_from_discard(deck)
print(f"{bob.name} draws a {drawn_card} from the discard pile.")

# Bob decides to replace a card in his hand with the drawn card based on the game logic
# For simplicity, let's replace the card with the highest value
highest_value_index = max(range(len(bob.hand)), key=lambda i: card_value_for_comparison(bob.hand[i]))
discarded_card = bob.exchange_card(highest_value_index, drawn_card)
deck.add_to_discard(discarded_card)

print(f"{bob.name} exchanges {discarded_card} for {drawn_card}.")
print(f"{bob.name}'s new hand: {bob.hand}")
print(f"Top of the discard pile: {deck.top_discard()}")



-- Bob's Turn --
Bob's initial hand: [Card(value='5', suit='♥', color='red'), Card(value='Q', suit='♠', color='black'), Card(value='7', suit='♥', color='red'), Card(value='9', suit='♣', color='black')]
Bob draws a Card: 8 of ♣ (black) from the discard pile.
Bob exchanges Card: Q of ♠ (black) for Card: 8 of ♣ (black).
Bob's new hand: [Card(value='5', suit='♥', color='red'), Card(value='8', suit='♣', color='black'), Card(value='7', suit='♥', color='red'), Card(value='9', suit='♣', color='black')]
Top of the discard pile: Card: Q of ♠ (black)


5. **End of Round:**

The round ends after each player has taken a turn.

scoring is calculated for each player based on the cards in their hand.

In [21]:
# Here we add the scores of both players, Aice and Bob and determine who wins this game:
alice_score = alice.calculate_score()
bob_score = bob.calculate_score()
print(f"{alice.name}'s score: {alice_score}")
print(f"{bob.name}'s score: {bob_score}")

if alice_score < bob_score:
    alice.is_winner = True
    print(f"{alice.name} wins!")
elif alice_score > bob_score:
    bob.is_winner = True
    print(f"{bob.name} wins!")

Alice's score: 19
Bob's score: 29
Alice wins!
